In [2]:
!pip install tensorflow

  Obtaining dependency information for tensorflow from https://files.pythonhosted.org/packages/3c/e3/e868f1d5951047f950d2ba1e04a765a3328a51f06996b67976d6102f8227/tensorflow-2.19.0-cp311-cp311-win_amd64.whl.metadata
  Obtaining dependency information for absl-py>=1.0.0 from https://files.pythonhosted.org/packages/f6/d4/349f7f4bd5ea92dab34f5bb0fe31775ef6c311427a14d5a5b31ecb442341/absl_py-2.2.2-py3-none-any.whl.metadata
  Obtaining dependency information for astunparse>=1.6.0 from https://files.pythonhosted.org/packages/2b/03/13dde6512ad7b4557eb792fbcf0c653af6076b81e5941d36ec61f7ce6028/astunparse-1.6.3-py2.py3-none-any.whl.metadata
  Obtaining dependency information for flatbuffers>=24.3.25 from https://files.pythonhosted.org/packages/b8/25/155f9f080d5e4bc0082edfda032ea2bc2b8fab3f4d25d46c1e9dd22a1a89/flatbuffers-25.2.10-py2.py3-none-any.whl.metadata
  Obtaining dependency information for gast!=0.5.0,!=0.5.1,!=0.5.2,>=0.2.1 from https://files.pythonhosted.org/packages/a3/61/8001b38461d751c

In [3]:
import tensorflow as tf 
import time
from tensorflow.keras import datasets, layers, models 
import matplotlib.pyplot as plt
import numpy as np

In [4]:
from tensorflow.keras import datasets

# Load and normalize CIFAR-10 dataset
(x_train, y_train), (x_test, y_test) = datasets.cifar10.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

# Check the shape
print(f'Training data shape: {x_train.shape}')
print(f'Testing data shape: {x_test.shape}')

170498071/170498071 ━━━━━━━━━━━━━━━━━━━━ 62s 0us/step 
Training data shape: (50000, 32, 32, 3)
Testing data shape: (10000, 32, 32, 3)


In [5]:
from tensorflow.keras import layers, models

def create_model():
    model = models.Sequential([
        layers.Conv2D(32, (3, 3), activation='relu', input_shape=(32, 32, 3)),
        layers.MaxPooling2D((2, 2)),
        layers.Conv2D(64, (3, 3), activation='relu'),
        layers.MaxPooling2D((2, 2)),
        layers.Flatten(),
        layers.Dense(64, activation='relu'),
        layers.Dense(10, activation='softmax')  
    ])
    return model

In [6]:
import time
import tensorflow as tf

def benchmark(device):
    with tf.device(device):
        model = create_model()
        model.compile(
            optimizer='adam',
            loss='sparse_categorical_crossentropy',
            metrics=['accuracy']
        )

        start_time = time.time()
        history = model.fit(
            x_train, y_train,
            epochs=5,
            batch_size=64,
            verbose=2,
            validation_data=(x_test, y_test)
        )
        end_time = time.time()

        print(f"\nTraining on {device} took {end_time - start_time:.2f} seconds")
        return history

In [7]:
# Benchmark training on CPU
history_cpu = benchmark('/CPU:0')

C:\Users\UMESH\anaconda3.x\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/5
782/782 - 24s - 31ms/step - accuracy: 0.4163 - loss: 1.6151 - val_accuracy: 0.5075 - val_loss: 1.3768
Epoch 2/5
782/782 - 25s - 32ms/step - accuracy: 0.5509 - loss: 1.2666 - val_accuracy: 0.5751 - val_loss: 1.1964
Epoch 3/5
782/782 - 24s - 31ms/step - accuracy: 0.6014 - loss: 1.1308 - val_accuracy: 0.6111 - val_loss: 1.1153
Epoch 4/5
782/782 - 21s - 27ms/step - accuracy: 0.6347 - loss: 1.0418 - val_accuracy: 0.6209 - val_loss: 1.0782
Epoch 5/5
782/782 - 18s - 23ms/step - accuracy: 0.6614 - loss: 0.9710 - val_accuracy: 0.6443 - val_loss: 1.0299

Training on /CPU:0 took 127.04 seconds


In [8]:
# Set TensorFlow to use GPU for training
history_gpu = benchmark('/GPU:0')

Epoch 1/5
782/782 - 21s - 27ms/step - accuracy: 0.4519 - loss: 1.5196 - val_accuracy: 0.5587 - val_loss: 1.2491
Epoch 2/5
782/782 - 18s - 23ms/step - accuracy: 0.5935 - loss: 1.1557 - val_accuracy: 0.6071 - val_loss: 1.1203
Epoch 3/5
782/782 - 19s - 25ms/step - accuracy: 0.6422 - loss: 1.0301 - val_accuracy: 0.6448 - val_loss: 1.0339
Epoch 4/5
782/782 - 18s - 22ms/step - accuracy: 0.6739 - loss: 0.9392 - val_accuracy: 0.6589 - val_loss: 0.9793
Epoch 5/5
782/782 - 18s - 24ms/step - accuracy: 0.6988 - loss: 0.8720 - val_accuracy: 0.6750 - val_loss: 0.9409

Training on /GPU:0 took 102.02 seconds
